<a href="https://colab.research.google.com/github/j0rdan0/AI-notebooks/blob/main/LLM_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install huggingface_hub
#!pip install -U BitsAndBytes

# authenticate to hugging face
def hf_auth():
  import huggingface_hub
  from google.colab import userdata

  hf_token = userdata.get('HF_TOKEN`')
  huggingface_hub.login(token=hf_token)

In [ ]:
# fetch the model
def get_model(model_name):
  from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig
  from accelerate.test_utils.testing import get_backend

  device,_,_ = get_backend()
  quantization_config = BitsAndBytesConfig(load_in_8bit=True)

  hf_auth()

  model = AutoModelForCausalLM.from_pretrained(model_name,device_map="auto", quantization_config=quantization_config,attn_implementation="flash_attention_2").to(device)
  model.generation_config.cache_implementation = "static" # set kv status cache

  model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

  return model


In [ ]:
# get tokenizer for model
def get_tokenizer(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name,padding_side="left")

# perform inference on the model
def send_prompt(prompt,model,tokenizer):
  inputs = tokenizer(prompt,return_tensors="pt").to("cuda")
  response = model.generate(**inputs,max_new_tokens=300,do_sample=True,num_beams=1)

  return tokenizer.batch_decode(response,skip_special_tokens=True)
